#setup
- connect to google drive in this step

In [1]:
import pandas as pd
import math
import numpy as np
import os
import glob
import sklearn
import pickle
import glob
import scipy.io
import cv2
from google.colab.patches import cv2_imshow

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
project_path = '/content/drive/MyDrive/demo-me-2021-07-14'
output_dir = os.path.join(project_path, "behaviors/")
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

In [4]:
def get_data(individual, bodypart, h5_file):
  mouse_data = h5_file.xs(individual,level='individuals',axis=1)
  out_data = mouse_data.xs(bodypart,level='bodyparts',axis=1)
  out_data.columns = out_data.columns.droplevel("scorer")
  output = out_data.copy()
  return output

In [5]:
def within_area(area_vector, input_coor):
  area_startx = area_vector[0]
  area_starty = area_vector[1]
  area_distx = area_vector[2]
  area_disty = area_vector[3]
  x = input_coor["x"].iloc[0]
  y = input_coor["y"].iloc[0]
  if (area_startx <= x <= (area_startx+area_distx)) and (area_starty <= y <= (area_starty+area_disty)):
    result = 1
  else:
    result = 0
  return result

In [6]:
def euclid_dist(point1_coor, point2_coor):
  point1 = np.array((point1_coor["x"].iloc[0], point1_coor["y"].iloc[0]))
  point2 = np.array((point2_coor["x"].iloc[0], point2_coor["y"].iloc[0]))
  output_dist = np.linalg.norm(point1 - point2)
  return output_dist

#on_male_side and putative mounts by disappearing female

Note: dlc origin is in the top left; x goes to the right, y goes down

In [ ]:
#for video in glob.glob(project_path + "/videos/" + "*.avi"):
for video in [project_path + "/videos/PZ70_1.avi"]:
  video_name = os.path.splitext(os.path.basename(video))[0]

  # area_vec = [area_startx, area_starty, area_distx, area_disty]
  female_side_mat = scipy.io.loadmat(project_path + "/videos/" + video_name + "_female_side.mat")
  female_side_vec = female_side_mat['croprect'][0]
  male_side_mat = scipy.io.loadmat(project_path + "/videos/" + video_name + "_male_side.mat")
  male_side_vec = male_side_mat['croprect'][0]

  h5_file = pd.read_hdf(project_path +'/videos/'+ video_name + 'DLC_dlcrnetms5_demoJul14shuffle0_20000_el_filtered.h5')
  [nframes, ncols] = h5_file.shape

  # get all data ready for accessing as needed later on
  mouse1_feature_points = {}
  mouse1_feature_points['mus1_snout'] = get_data('mus1', 'snout', h5_file)
  mouse1_feature_points['mus1_shoulder'] = get_data('mus1', 'shoulder', h5_file)
  mouse1_feature_points['mus1_spine1'] = get_data('mus1', 'spine1', h5_file)
  mouse1_feature_points['mus1_spine2'] = get_data('mus1', 'spine2', h5_file)
  mouse1_feature_points['mus1_spine3'] = get_data('mus1', 'spine3', h5_file)
  mouse1_feature_points['mus1_spine4'] = get_data('mus1', 'spine4', h5_file)
  mouse1_feature_points['mus1_tailbase'] = get_data('mus1', 'tailbase', h5_file)

  mouse2_feature_points = {}
  mouse2_feature_points['mus2_snout'] = get_data('mus2', 'snout', h5_file)
  mouse2_feature_points['mus2_shoulder'] = get_data('mus2', 'shoulder', h5_file)
  mouse2_feature_points['mus2_spine1'] = get_data('mus2', 'spine1', h5_file)
  mouse2_feature_points['mus2_spine2'] = get_data('mus2', 'spine2', h5_file)
  mouse2_feature_points['mus2_spine3'] = get_data('mus2', 'spine3', h5_file)
  mouse2_feature_points['mus2_spine4'] = get_data('mus2', 'spine4', h5_file)
  mouse2_feature_points['mus2_tailbase'] = get_data('mus2', 'tailbase', h5_file)

  mouse3_feature_points = {}
  mouse3_feature_points['mus3_snout'] = get_data('mus3', 'snout', h5_file)
  mouse3_feature_points['mus3_shoulder'] = get_data('mus3', 'shoulder', h5_file)
  mouse3_feature_points['mus3_spine1'] = get_data('mus3', 'spine1', h5_file)
  mouse3_feature_points['mus3_spine2'] = get_data('mus3', 'spine2', h5_file)
  mouse3_feature_points['mus3_spine3'] = get_data('mus3', 'spine3', h5_file)
  mouse3_feature_points['mus3_spine4'] = get_data('mus3', 'spine4', h5_file)
  mouse3_feature_points['mus3_tailbase'] = get_data('mus3', 'tailbase', h5_file)

  def torso_in_area(area_vector, mouse_num, i):
    # finds if any part of torso at all is in area
    if mouse_num == 1:
      feature_points = mouse1_feature_points
      name = 'mus1'
    elif mouse_num == 2:
      feature_points = mouse2_feature_points
      name = 'mus2'
    elif mouse_num == 3:
      feature_points = mouse3_feature_points
      name = 'mus3'

    snout_result = within_area(area_vector, feature_points[name + '_snout'].loc[[i]])
    shoulder_result = within_area(area_vector, feature_points[name + '_shoulder'].loc[[i]])
    spine1_result = within_area(area_vector, feature_points[name + '_spine1'].loc[[i]])
    spine2_result = within_area(area_vector, feature_points[name + '_spine2'].loc[[i]])
    spine3_result = within_area(area_vector, feature_points[name + '_spine3'].loc[[i]])
    spine4_result = within_area(area_vector, feature_points[name + '_spine4'].loc[[i]])
    tailbase_result = within_area(area_vector, feature_points[name + '_tailbase'].loc[[i]])

    if snout_result==1 or shoulder_result==1 or spine1_result==1 or spine2_result==1 or spine3_result==1 or spine4_result==1 or tailbase_result==1:
      result = 1
    else:
      result = 0
    return result

  behavior = pd.DataFrame(index=range(nframes),columns=['behavior'])
  behavior_events = pd.DataFrame(index=range(nframes - 1), columns=['events_index', 'minutes', 'seconds', 'behavior_events'])

  for i in range(nframes):
    if torso_in_area(female_side_vec, 1, i) or torso_in_area(female_side_vec, 2, i) or torso_in_area(female_side_vec, 3, i):
      event_name = 'female_side'

    elif (torso_in_area(male_side_vec, 1, i) and torso_in_area(male_side_vec, 2, i)) or \
      (torso_in_area(male_side_vec, 1, i) and torso_in_area(male_side_vec, 3, i)) or \
      (torso_in_area(male_side_vec, 2, i) and torso_in_area(male_side_vec, 3, i)):
        event_name = 'male_side'

    else:
      event_name = 'mount'

    behavior["behavior"][i] = event_name

  events_n = 0
  for i in range(1, nframes):
    if behavior["behavior"][i] != behavior["behavior"][i-1]:
      behavior_events["events_index"][events_n] = i
      behavior_events["minutes"][events_n] = i / (30*60)
      behavior_events["seconds"][events_n] = round(i / 30) % 60
      behavior_events["behavior_events"][events_n] = behavior["behavior"][i]
      events_n = events_n + 1
  print(events_n)
  behavior_events = behavior_events.dropna(how='all')

  with open(output_dir + video_name + '_behavior.pickle', 'wb') as f:
      pickle.dump(behavior, f)
  with open(output_dir + video_name + '_behavior_events.pickle', 'wb') as f:
      pickle.dump(behavior_events, f)

667


In [ ]:
#print(behavior["behavior"][328])
#print(mouse1_feature_points['mus1' + '_shoulder'].loc[[328]])
#print(mouse3_feature_points['mus3' + '_shoulder'].loc[[328]])

male_side
coords        x        y  likelihood
328     249.195  113.587       0.915
coords        x        y  likelihood
328     249.195  113.587       0.915


Behavior categorization logic (behavior_postprocess version):
- if mouse body detected on female side, -> definitely female_side

- if no mouse detected on female side, -> putative male_side
 - problem: when grooming, sometimes does not detect lone female on female side

- if no mouse detected on female side, only 1 mouse detected on male side -> putative mount to be checked by hand
 - problem: sometimes putative mounts are too quick to really be mounts
 - problem: sometimes undetected female_side

- so if no mouse detected on female side, both mice detected on male side -> male_side

at this point:
- 'female_side' means mouse detected on female side
- 'male_side' means not 1 (so 0 or 2) mouse back detected on male side and no mice on female side
- 'mount' means 1 mouse back detected on male side and no mice on female side

#make labelled behavior video

In [7]:
project_path = '/content/drive/MyDrive/demo-me-2021-07-14/'
output_dir = os.path.join(project_path, "behaviors/")
if not os.path.exists(output_dir):
  os.mkdir(output_dir)

end_frame = 36000
for video in [project_path + "/videos/PZ71_1.avi"]:
#for video in glob.glob(project_path + "/videos/" + "*.avi"):
  video_name = os.path.splitext(os.path.basename(video))[0]
  
  labeled_video = video_name+'DLC_dlcrnetms5_demoJul14shuffle0_20000_el_filtered_id_labeled.mp4'

  behavior_file = open(output_dir + video_name + '_behavior.pickle', 'rb')
  behavior = pickle.load(behavior_file)
  behavior_file.close()

  cap = cv2.VideoCapture(project_path + 'videos/' + labeled_video)

  # Check if camera opened successfully
  if (cap.isOpened()== False): 
    print("Error opening video stream or file")

  while cap.isOpened():
    ret,frame = cap.read()
    print(frame.shape)
    frame_width = int(frame.shape[1])
    frame_height = int(frame.shape[0])
    break

  output_video = output_dir + video_name + '_behavior_video.mp4'

  out = cv2.VideoWriter(output_video,cv2.VideoWriter_fourcc(*'MP4V'), 30, (frame_width,frame_height))

  count = 0
  while cap.isOpened():
    ret,frame = cap.read()
    current_behavior = behavior["behavior"][count]
    text_color = (0,0,0)
    if current_behavior == 'male_side':
      text_color = (0, 0, 150)
    elif current_behavior == 'mount':
      text_color = (0, 0, 255)
    elif current_behavior == 'female_side':
      text_color = (50, 0, 0)
    elif current_behavior == 'forest_mount':
      text_color = (0, 255, 0)
    elif current_behavior == 'unclear':
      text_color = (0, 255, 0)
    else:
      print(current_behavior)
      print(count)
    cv2.putText(frame, str(current_behavior), (25, 25), cv2.FONT_HERSHEY_DUPLEX, 1, text_color, 2)
    cv2.putText(frame, str(count), (25, frame_height), cv2.FONT_HERSHEY_DUPLEX, 1, (0, 0, 0), 2)
    out.write(frame)

    count = count + 1
    if count > end_frame:
      break
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break

  cap.release()
  out.release()

(166, 286, 3)
nan
632
nan
1901
nan
1913
nan
1961
nan
1962
nan
1967
nan
1970
nan
2018
nan
2019
nan
2020
nan
2021
nan
2094
nan
3603
nan
3604
nan
3605
nan
3606
nan
4259
nan
4260
nan
4261
nan
6966
nan
6967
nan
6968
nan
6969
nan
7841
nan
8960
nan
11040
nan
11042
nan
11043
nan
11044
nan
11045
nan
11047
nan
11048
nan
11049
nan
11050
nan
11124
nan
11126
nan
11127
nan
11130
nan
11131
nan
11132
nan
11133
nan
11137
nan
12502
nan
12503
nan
13623
nan
13624
nan
13625
nan
13626
nan
13629
nan
13675
nan
13679
nan
13680
nan
13681
nan
13683
nan
13699
nan
14219
nan
14222
nan
14224
nan
14225
nan
14226
nan
14227
nan
14228
nan
14229
nan
14230
nan
14231
nan
14235
nan
14236
nan
14252
nan
14254
nan
14586
nan
14881
nan
14882
nan
14883
nan
14884
nan
15153
nan
15154
nan
15155
nan
15156
nan
15157
nan
15160
nan
15161
nan
15162
nan
15163
nan
15164
nan
15392
nan
15393
nan
15398
nan
15399
nan
15400
nan
15401
nan
15448
nan
15449
nan
16667
nan
16669
nan
16694
nan
16695
nan
16696
nan
16701
nan
16708
nan
16710
nan
16713
na

#read previous behavior data pickles

In [ ]:
video_name = 'PZ80_2'

behavior_file = open(output_dir + video_name + '_behavior.pickle', 'rb')
behavior = pickle.load(behavior_file)
behavior_file.close()

behavior_events_file = open(output_dir + video_name + '_behavior_events.pickle', 'rb')
behavior_events = pickle.load(behavior_events_file)
behavior_events_file.close()

print(video_name)
first_20_min = behavior.truncate(after=(36000 - 1)).copy()
male_side_seconds = (first_20_min[first_20_min["behavior"] == 'male_side']["behavior"].count() + first_20_min[first_20_min["behavior"] == 'mount']["behavior"].count()) / 30
male_side_percent = male_side_seconds / (36000 / 30)
mount_seconds = first_20_min[first_20_min["behavior"] == 'mount']["behavior"].count() / 30

first_20_min_events = behavior_events.copy()
male_side_entrances = first_20_min_events[first_20_min_events["behavior_events"] == 'male_side']["behavior_events"].count()
mount_events = first_20_min_events[first_20_min_events["behavior_events"] == 'mount']["behavior_events"].count()

print('male_side seconds: ', male_side_seconds)
print('male_side percent: ', male_side_percent)
print('mount seconds: ', mount_seconds)
print('mount events: ', mount_events)

PZ80_2
male_side seconds:  418.43333333333334
male_side percent:  0.3486944444444445
mount seconds:  17.966666666666665
mount events:  15


In [ ]:
print(first_20_min_events[first_20_min_events["behavior_events"] == 'mount'])

    events_index    minutes seconds behavior_events
1            583   0.323889      19           mount
3           1365   0.758333      46           mount
67         11516   6.397778      24           mount
94         13085   7.269444      16           mount
96         13160   7.311111      19           mount
98         13266       7.37      22           mount
125        14778       8.21      13           mount
127        14868       8.26      16           mount
129        15019   8.343889      21           mount
160        21250  11.805556      48           mount
224        31615  17.563889      34           mount
226        33563  18.646111      39           mount
233        36671  20.372778      22           mount
235        37245  20.691667      42           mount
237        37724  20.957778      57           mount
